In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import requests
from bs4 import BeautifulSoup

In [2]:
riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.top_summoners
matches_collection = loladb.top_matches
matches_checked = loladb.top_matches_checked

if summoners_collection.count() == 0:
    #Save summoner information to mongodb
    challengers = riotapi.get_challenger()
    summoners_collection.insert_many([json.loads(entry.to_json()) for entry in challengers.data.entries])
    masters = riotapi.get_master()
    summoners_collection.insert_many([json.loads(entry.to_json()) for entry in masters.data.entries])

In [3]:
print summoners_collection.count()
print matches_collection.count()

390
179


In [7]:
#check the summoners involved in current collected top matches
from sets import Set
involvedSummoners = Set()
for a_match in matches_collection.find():
    participants = a_match['participantIdentities']
    for a_par in participants:
        involvedSummoners.add(a_par['player']['summonerName'])
print len(involvedSummoners)

384


In [35]:
def GetTierFromOPGG(summoner_name):
    summonerURL = "http://www.op.gg/summoner/userName=" + summoner_name
    print summonerURL
    result = requests.get(summonerURL)
    soup = BeautifulSoup(result.content, 'html.parser')
    tier_part = soup.find("div", { "class" : "TierRank" })
    if tier_part == None:
        return None, None
    
    tier_text = tier_part.getText()
    #print tier_text
    tier_splits = tier_text.split()
    tier = tier_splits[0]
    division = ""
    #print len(tier_splits)
    if len(tier_splits) > 1:
        division = tier_splits[1]
    
    return tier,division

In [14]:
GetTierFromOPGG(summoner_0['playerOrTeamName'])

http://www.op.gg/summoner/userName=유내현

Challenger

1
Challenger-


In [5]:
topIds = []
for a_summoner in summoners_collection.find():
    topIds.append(a_summoner['playerOrTeamId'])
print topIds

[u'12056399', u'6993034', u'50890124', u'5122177', u'2044235', u'16140220', u'45311145', u'10030815', u'50261037', u'3964535', u'49820338', u'49040956', u'1881895', u'9221937', u'21934593', u'41090177', u'2481136', u'25260179', u'12410600', u'3263292', u'43070124', u'12182163', u'39050997', u'5134464', u'9100379', u'5657821', u'21650926', u'1317404', u'3412304', u'8968873', u'1093349', u'14051535', u'32200268', u'4005106', u'22071749', u'5144946', u'4026603', u'11705657', u'50651153', u'20160399', u'8834525', u'1754593', u'50890125', u'7938280', u'13802496', u'50980129', u'3166512', u'17596207', u'8968235', u'2909230', u'2000928', u'8301731', u'27251248', u'3979503', u'5290661', u'7695143', u'45561472', u'7735748', u'5992425', u'26691029', u'4431058', u'50791988', u'3964487', u'50820240', u'39710916', u'30941014', u'8342164', u'8008857', u'25181100', u'49380619', u'44990245', u'18451556', u'43410136', u'35801706', u'49190125', u'1164825', u'49730723', u'6140824', u'2476861', u'52240735

In [29]:
print match_list[0].to_json()

{
    "champion": 43, 
    "lane": "MID", 
    "matchId": 2666807225, 
    "platformId": "KR", 
    "queue": "TEAM_BUILDER_RANKED_SOLO", 
    "role": "SOLO", 
    "season": "PRESEASON2017", 
    "timestamp": 1482217860145
}


In [17]:
for a_idx, a_summoner in enumerate(summoners_collection.find()):
    print 'Checking summoner ' + str(a_summoner['playerOrTeamId'])
    if a_idx == 21:
        break

Checking summoner 12056399
Checking summoner 6993034
Checking summoner 50890124
Checking summoner 5122177
Checking summoner 2044235
Checking summoner 16140220
Checking summoner 45311145
Checking summoner 10030815
Checking summoner 50261037
Checking summoner 3964535
Checking summoner 49820338
Checking summoner 49040956
Checking summoner 1881895
Checking summoner 9221937
Checking summoner 21934593
Checking summoner 41090177
Checking summoner 2481136
Checking summoner 25260179
Checking summoner 12410600
Checking summoner 3263292
Checking summoner 43070124
Checking summoner 12182163


In [ ]:
import os.path
last_summoner_id = -1
if os.path.exists("LastTopSummonerId.txt"):
    with open("LastTopSummonerId.txt", "r") as f:
        last_summoner_id = int(f.readline())
        print "Read last summoner id " + str(last_summoner_id)
        
def IsSoloRank(match_type):
    if match_type == 'TEAM_BUILDER_RANKED_SOLO' or match_type == 'TEAM_BUILDER_DRAFT_RANKED_5x5':
        return True
    else:
        return False

def RecordTopMatchForSummoner(a_summoner):
    global last_summoner_id
    if last_summoner_id != -1:
        if last_summoner_id != int(a_summoner['playerOrTeamId']):
            return
        else:
            last_summoner_id = -1
    
    with open("LastTopSummonerId.txt", "w") as f:
        print "Write last top summoner id " + str(a_summoner['playerOrTeamId'])
        f.write(str(a_summoner['playerOrTeamId']))
        f.close()
        
    summoner = riotapi.get_summoner_by_id(a_summoner['playerOrTeamId'])
    match_list = summoner.match_list()
    list_size = len(match_list)

    match_count = 0
    tier_array = []

    for match_idx, match in enumerate(match_list):
        #print 'Checking %d of %d' % (match_idx, list_size)

        match_record = matches_checked.find_one({'matchId':match.id})
        if match_record != None:
            if match_record['shouldStop'] == False:
                continue
            else:
                break

        matches_checked.insert_one({'matchId':match.id, 'shouldStop':False})

        if IsSoloRank(match.data.queue) == False:
            print 'Continue for not being a solo rank with id ' + str(match.id)
            continue

        match_data = match.match()
        a_match = json.loads(match_data.to_json())
        if a_match['matchVersion'].startswith('6.24') == False:
            print 'Break for not being in 6.24 with id ' + str(match.id)
            matches_checked.update_one({'matchId':match.id}, {'$set': {'shouldStop': True}})
            break

        a_duration = a_match['matchDuration']
        a_minute = a_duration/60
        if a_minute < 20:
            print 'Continue for not being a full game with id ' + str(match.id)
            continue 

        match_count += 1

        isTopMatch = True

        participants = a_match['participants'] 
        for a_participant_idx,a_participantIdentity in enumerate(a_match['participantIdentities']):
            a_id = a_participantIdentity['player']['summonerId']
            isTopMatch = str(a_id) in topIds

            if isTopMatch == False:
                print 'Break for not being a top player ' + str(a_id)
                break

        if isTopMatch == True:
            if matches_collection.find_one({'matchId':match.id}) == None:
                matches_collection.insert_one(a_match)
                print 'A match saved'

for a_idx, a_summoner in enumerate(summoners_collection.find()):
    print 'Checking summoner ' + str(a_idx)
    RecordTopMatchForSummoner(a_summoner)

Read last summoner id 43070124
Checking summoner 0
Checking summoner 1
Checking summoner 2
Checking summoner 3
Checking summoner 4
Checking summoner 5
Checking summoner 6
Checking summoner 7
Checking summoner 8
Checking summoner 9
Checking summoner 10
Checking summoner 11
Checking summoner 12
Checking summoner 13
Checking summoner 14
Checking summoner 15
Checking summoner 16
Checking summoner 17
Checking summoner 18
Checking summoner 19
Checking summoner 20
Write last top summoner id 43070124
https://kr.api.pvp.net/api/lol/kr/v2.2/matchlist/by-summoner/43070124?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://kr.api.pvp.net/api/lol/kr/v2.2/match/2659166457?includeTimeline=True&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://kr.api.pvp.net/api/lol/kr/v1.4/summoner/2769362,3429822?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Break for not being a top player 2769362
https://kr.api.pvp.net/api/lol/kr/v2.2/match/2659164559?includeTimeline=True&api_key=79428a9e-5d98-469b-9b9b-429c1a

In [7]:
for m in matches_collection.find():
    print m['matchId']

2663078436
2662977693
2661320358
2657543040
2657445323
2654238849
2667868430
2667836310
2666356306
2666259469
2667674210
2667384757
2666169873
2666187242
2666137075
2666086452
2666063943
2665243196
2664061376
2663939590
2663200364
2662957760
2660875230
2660744089
2659922153
2659084576
